In [1]:
#import library

import cv2 as cv #for image processing
import time #for current time control
import numpy as np
print(cv.__version__)

4.5.4-dev


In [2]:
#obtain camera meta-information

capture = cv.VideoCapture('2021.11.23.10.31.37_figure.avi') #open camera
ret, frame = capture.read() #read camera's input
width = capture.get(cv.CAP_PROP_FRAME_WIDTH) #obtain width
height = capture.get(cv.CAP_PROP_FRAME_HEIGHT) #obtain height
fps = capture.get(cv.CAP_PROP_FPS) #obtain fps
#fps=8
#configuration of RGB recoding

fourcc = cv.VideoWriter_fourcc(*'DIVX') #for codec
#out = cv.VideoWriter(filename, fourcc, fps, (int(width), int(height))) #class for recoding
print('width {0}, height {1}, fps {2}'.format(width, height, fps)) #print information

width 640.0, height 480.0, fps 30.0


In [3]:
class mouse:
    def __init__(self):
        self.position=None #
        self.kind=None #mother or baby
        self.ID=None #RFID
        self.center = None
    
    def update(self):
        pass

class enviroment:
    def __init__(self):
        self.tracking_center=[]
        
    

In [4]:
#main body. it describes all task for each frame.
en = enviroment()
mother = mouse()
baby1 = mouse()

start_time=time.time() #obtain current time for basis
start_time_=time.time()
area_threshold=500
area_threshold_baby=300
flag = 1 #flag for record
delay = int(1000/fps) #delay based on fps
win_name = 'Tracking APIs'
#print(int(time.time()-start_time))
count=0
while True: #repeat infinitely
    count+=1
    if int(time.time()-start_time_)==1:
        print(count)
        start_time_ = time.time()
        count=0
    _, frame_original = capture.read() #read a frame
    frame = cv.inRange(frame_original, (0,0,0), (20,20,20))
    frame[0:25] = 0
    frame[:,:80] = 0
    frame[:,600:] = 0
    
    frame_baby = cv.inRange(frame_original, (20, 20, 20), (70,70,120))
    frame_baby = cv.dilate(frame_baby, (5,5), iterations = 2)
    frame_baby[0:25] = 0
    frame_baby[:,:100] = 0
    frame_baby[:,600:] = 0
    contour, _ = cv.findContours(frame, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    contour2, _ = cv.findContours(frame_baby, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    img_draw = frame_original.copy()
    
    for cnt in contour:
        if cv.contourArea(cnt) > area_threshold:
            M = cv.moments(cnt, False)
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
            cv.circle(img_draw, (cX, cY), 3, (255, 0, 0), -1)
            cv.drawContours(img_draw, cnt, -1, (0,255,0), 2)
            en.tracking_center.append((cX,cY))
            break
            
    for cnt in contour2:
        if cv.contourArea(cnt) > area_threshold_baby:
            #print("tt")
            cv.drawContours(img_draw, cnt, -1, (0,0,255), 2)
            #break
   
    if len(en.tracking_center)>=2:
        for i in range(int(len(en.tracking_center)-1)):
            cv.line(img_draw, en.tracking_center[i], en.tracking_center[i+1], (255,0,0))
        if len(en.tracking_center)==200:
            del en.tracking_center[0]
                        
    current_time=[time.localtime().tm_year,time.localtime().tm_mon, \
                  time.localtime().tm_mday, time.localtime().tm_hour, time.localtime().tm_min, \
                  time.localtime().tm_sec]   #obtain current time
    current_time_second=time.time()

    cv.imshow(win_name, img_draw)
    cv.imshow("er",frame_baby)
    cv.imshow("mother",frame)
    
    if int(current_time_second-start_time) % 3600 == 0 and flag == 1: #per 1 hour re-name video file.
        filename =str(current_time[0])+"."+str(current_time[1])+"."+str(current_time[2])+"."+str(current_time[3])+"."+\
                                       str(current_time[4])+"."+str(current_time[5])
        out = cv.VideoWriter(filename+'_figure'+'.avi', fourcc, fps, (int(width), int(height))) #class for recoding
        out_raw = cv.VideoWriter(filename+'.avi', fourcc, fps, (int(width), int(height))) #class for recoding
        flag=0    
    elif int(current_time_second-start_time) % 3599 == 0 and int(current_time_second-start_time)!=0: #check 1 hour
        flag=1

    out.write(img_draw) # write a frame in a video 
    out_raw.write(frame_original)
    
    key = cv.waitKey(15)  # if ESC key is pressed, then exit 
    if key ==27:
        break

    
capture.release()  #terminate all the code.
cv.destroyAllWindows()



30
29
29
30
28
31
31
32
30
30
25
26
30
24
30
28
29
27
25
25
28
30
29
28
25
27
25
28
30
31
32
28
31
31
29
30
30
30
26
30
27
28
30
27
31
25
27
28
25
28
28
31
29
26
29
26
28
25
28
31
29
31
32
29
26
31


KeyboardInterrupt: 